In [1]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [23]:
# Constants
grid_size = 5
spacing_xy = 48.8
spacing_z = 27.112
half_shift = spacing_xy / 4

# Board ID mapping
board_ids = {
    0: 'E1',   1: 'O10', 2: 'E2',  3: 'O2',  4: 'E3',
    5: 'O3',   6: 'E9',  7: 'O9',  8: 'E5',  9: 'O5',
    10: 'E6', 11: 'O6', 12: 'E7', 13: 'O7', 14: 'E8'
}

# CAEN-to-layer mapping
caen_mapping = {
    0: [(0, 24, 0), (25, 49, 1), (50, 62, '4a')],
    1: [(0, 24, 2), (25, 49, 3), (50, 61, '4b')],
    2: [(0, 24, 5), (25, 49, 6), (50, 62, '9a')],
    3: [(0, 24, 7), (25, 49, 8), (50, 61, '9b')],
    4: [(0, 24, 10), (25, 49, 11), (50, 62, '14a')],
    5: [(0, 24, 12), (25, 49, 13), (50, 61, '14b')],
}

records = []

for caen, ranges in caen_mapping.items():
    for start_ch, end_ch, layer in ranges:
        num_channels = end_ch - start_ch + 1

        if isinstance(layer, str) and layer.endswith('a'):
            position_offset = 0
            layer_number = int(layer[:-1])
        elif isinstance(layer, str) and layer.endswith('b'):
            position_offset = 13
            layer_number = int(layer[:-1])
        else:
            position_offset = 0
            layer_number = layer

        for i in range(num_channels):
            caen_channel = start_ch + i
            position = position_offset + i

            # Grid position
            row = position // grid_size
            col = position % grid_size

            # Centered x,y
            x = -(2 - col) * spacing_xy
            y = (row - 2) * spacing_xy

            # Apply shift for staggered layers
            if layer_number % 2 == 0:
                x += half_shift
                y += half_shift
            else:
                x -= half_shift
                y -= half_shift

            z = layer_number * spacing_z
            channel_num = caen_channel + 64 * caen
            board_id = board_ids[layer_number]

            records.append({
                'caen': caen,
                'caen_channel': caen_channel,
                'Ch#': channel_num,
                'layer': layer_number,
                'brd_channel': position,
                'x': x,
                'y': y,
                'z': z,
                'board_id': board_id
            })

# Create DataFrame
df = pd.DataFrame.from_records(records)

# Optional: sort by Ch# or any other field
df = df.sort_values(by='Ch#').reset_index(drop=True)

print(df)


     caen  caen_channel  Ch#  layer  brd_channel      x      y        z  \
0       0             0    0      0            0  -85.4  -85.4    0.000   
1       0             1    1      0            1  -36.6  -85.4    0.000   
2       0             2    2      0            2   12.2  -85.4    0.000   
3       0             3    3      0            3   61.0  -85.4    0.000   
4       0             4    4      0            4  109.8  -85.4    0.000   
..    ...           ...  ...    ...          ...    ...    ...      ...   
370     5            57  377     14           20  -85.4  109.8  379.568   
371     5            58  378     14           21  -36.6  109.8  379.568   
372     5            59  379     14           22   12.2  109.8  379.568   
373     5            60  380     14           23   61.0  109.8  379.568   
374     5            61  381     14           24  109.8  109.8  379.568   

    board_id  
0         E1  
1         E1  
2         E1  
3         E1  
4         E1  
..       

In [18]:
df.to_pickle("geometry.pkl")

In [24]:
for i in range(len(df)):
    print(str(df['Ch#'][i]) +" "+str(df['layer'][i]) +" "+str(df['x'][i]) + " " + str(df['y'][i]))

0 0 -85.39999999999999 -85.39999999999999
1 0 -36.599999999999994 -85.39999999999999
2 0 12.2 -85.39999999999999
3 0 61.0 -85.39999999999999
4 0 109.8 -85.39999999999999
5 0 -85.39999999999999 -36.599999999999994
6 0 -36.599999999999994 -36.599999999999994
7 0 12.2 -36.599999999999994
8 0 61.0 -36.599999999999994
9 0 109.8 -36.599999999999994
10 0 -85.39999999999999 12.2
11 0 -36.599999999999994 12.2
12 0 12.2 12.2
13 0 61.0 12.2
14 0 109.8 12.2
15 0 -85.39999999999999 61.0
16 0 -36.599999999999994 61.0
17 0 12.2 61.0
18 0 61.0 61.0
19 0 109.8 61.0
20 0 -85.39999999999999 109.8
21 0 -36.599999999999994 109.8
22 0 12.2 109.8
23 0 61.0 109.8
24 0 109.8 109.8
25 1 -109.8 -109.8
26 1 -61.0 -109.8
27 1 -12.2 -109.8
28 1 36.599999999999994 -109.8
29 1 85.39999999999999 -109.8
30 1 -109.8 -61.0
31 1 -61.0 -61.0
32 1 -12.2 -61.0
33 1 36.599999999999994 -61.0
34 1 85.39999999999999 -61.0
35 1 -109.8 -12.2
36 1 -61.0 -12.2
37 1 -12.2 -12.2
38 1 36.599999999999994 -12.2
39 1 85.39999999999999 -12